## AutoGen Core

This is agnostic to the underlying Agent framework

You can use AutoGen AgentChat, or you can use something else; it's an Agent interaction framework.

From that point of view, it's positioned similarly to LangGraph.

#### The fundamental principle

[Autogen Core](https://microsoft.github.io/autogen/stable//user-guide/core-user-guide/quickstart.html) decouples an agent's logic from how messages are delivered.  
The framework provides a communication infrastructure, along with agent lifecycle, and the agents are responsible for their own work.

The communication infrastructure is called a `Agent Runtime`.

There are 2 types:
- **[Standalone](https://microsoft.github.io/autogen/stable//user-guide/core-user-guide/core-concepts/architecture.html#standalone-agent-runtime)** - Standalone runtime is suitable for single-process applications where all agents are implemented in the same programming language and running in the same process
- **[Distributed](https://microsoft.github.io/autogen/stable//user-guide/core-user-guide/core-concepts/architecture.html#distributed-agent-runtime)** - Distributed runtime is suitable for multi-process applications where agents may be implemented in different programming languages and running on different machines.

This file focuses on standalone runtime: the **SingleThreadedAgentRuntime**, a local embedded agent runtime implementation.


In [1]:
# Loading up required libraries
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_core import SingleThreadedAgentRuntime
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

from dotenv import load_dotenv
from dataclasses import dataclass

In [2]:
# loading up environment variables from env file
load_dotenv(override=True)

True

#### Defining Message object

This defines the structure we want for messages in our Agent framework.

In [3]:
# Simple dataclass

@dataclass
class Message:
    content: str

#### Defining our Agent

A subclass of RoutedAgent.

Every Agent has an **Agent ID** which has 2 components:  
- `agent.id.type` describes the kind of agent it is  
- `agent.id.key` gives it its unique identifier

Any method with the `@message_handler` decorated will have the opportunity to receive messages.


In [4]:
class SimpleAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("A Simple Agent")

    @message_handler
    async def on_message(self, message: Message, ctx: MessageContext) -> Message:
        return Message(content=f"This is {self.id.type}-{self.id.key}. You said {message.content}.")

#### Creating a Standalone runtime and register our agent type

In [5]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())

AgentType(type='simple_agent')

#### Starting runtime and sending a message

In [6]:
runtime.start()

In [7]:
agent_id = AgentId("simple_agent", "default")
response = await runtime.send_message(Message("Hi there! How can I learn agentic ai?"), agent_id)
print(">>>", response.content)

>>> This is simple_agent-default. You said Hi there! How can I learn agentic ai?.


#### Remember to always stop and close the runtime

In [8]:
await runtime.stop()
await runtime.close()

#### Something more interesting!

Using an AgentChat Assistant!

In [9]:
class LLMAgent(RoutedAgent):
    def __init__(self) -> None:
        super().__init__("LLMAgent")
        gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash")
        self._delegate = AssistantAgent("LLMAgent", model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        print(f"{self.id.type} received message: {message.content}")
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        reply = response.chat_message.content
        print(f"{self.id.type} responded: {reply}")
        return Message(content=reply)

In [10]:
runtime = SingleThreadedAgentRuntime()
await SimpleAgent.register(runtime, "simple_agent", lambda: SimpleAgent())
await LLMAgent.register(runtime, "LLMAgent", lambda: LLMAgent())

AgentType(type='LLMAgent')

In [14]:
runtime.start()
response = await runtime.send_message(Message("How can I learn agentic ai?"), AgentId("LLMAgent", "default"))
print(">>>", response.content)
response =  await runtime.send_message(Message(response.content), AgentId("simple_agent", "default"))
print(">>>", response.content)
response = await runtime.send_message(Message(response.content), AgentId("LLMAgent", "default"))

LLMAgent received message: How can I learn agentic ai?
LLMAgent responded: Learning about agentic AI is a fascinating and rapidly evolving field! Here's a breakdown of how you can learn, covering different approaches and resources:

**1. Understand the Fundamentals:**

*   **Artificial Intelligence (AI) Basics:**
    *   Start with the core concepts of AI: What is AI, machine learning (ML), deep learning (DL), natural language processing (NLP), etc.?
    *   **Resources:**
        *   **Online Courses:** Coursera, edX, Udacity, and fast.ai offer introductory AI and ML courses.
        *   **Books:** "Artificial Intelligence: A Modern Approach" by Stuart Russell and Peter Norvig is a classic textbook. "Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow" by Aurélien Géron is a practical guide.

*   **Agent Theory:**
    *   Learn about the theoretical foundations of agents: What defines an agent, its environment, goals, and how it perceives and acts. Understand concepts like

#### Always remember to stop and close the runtime. This also applies if you want to re-run the cell above.

In [16]:
await runtime.stop()
await runtime.close()

RuntimeError: Runtime is not started

#### Introducing game player agents - having 3 agents interact!

In [61]:
from pyexpat import model


class PlayerOneAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        gemini_client = OpenAIChatCompletionClient(model="gemini-1.5-flash", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

class PlayerTwoAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)

In [62]:
JUDGE = "You are judging a game of rock, paper, scissors. The players have made these choices:\n"

class RockPaperScissorsAgent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        gemini_client = OpenAIChatCompletionClient(model="gemini-2.0-flash", temperature=1.0)
        self._delegate = AssistantAgent(name, model_client=gemini_client)

    @message_handler
    async def handle_message_type(self, message: Message, ctx: MessageContext) -> Message:
            instruction = "You are playing rock, paper, scissors. Respond only with the one word, one of the following: rock, paper, or scissors."
            message = Message(content=instruction)
            player_one = AgentId("player_one", "default")
            player_two = AgentId("player_two", "default")
            response1 = await self.send_message(message, player_one)
            response2 = await self.send_message(message, player_two)
            result = f"Player 1: {response1.content}\nPlayer 2: {response2.content}\n"
            judgement = f"{JUDGE}{result}Who wins?"
            message = TextMessage(content=judgement, source="user")
            response = await self._delegate.on_messages([message], ctx.cancellation_token)
            return Message(content=result + response.chat_message.content)


In [63]:
runtime = SingleThreadedAgentRuntime()
await PlayerOneAgent.register(runtime, "player_one", lambda: PlayerOneAgent("player_one"))
await PlayerTwoAgent.register(runtime, "player_two", lambda: PlayerTwoAgent("player_two"))
await RockPaperScissorsAgent.register(runtime, "rock_paper_scissors", lambda: RockPaperScissorsAgent("rock_paper_scissors"))
runtime.start()

In [64]:
agent_id = AgentId("rock_paper_scissors", "default")
message = Message(content="go")
response = await runtime.send_message(message, agent_id)
print(response.content)

Player 1: rock
TERMINATE

Player 2: rock

It's a tie!
TERMINATE



In [65]:
await runtime.stop()
await runtime.close()